# Pathway Enrichment on the COVID-19 Knowledge Graph
This notebook outlines a standard pathway enrichment based on the nodes presented in the KG.

In [1]:
import sys
import time

import pandas as pd
import numpy as np
import scipy
from scipy.stats import fisher_exact
from statsmodels.stats.multitest import multipletests
import covid19kg
import pybel

# Notebook Provenance
The explicit display of time of execution and the versions of the software packages used.

In [2]:
sys.version

'3.7.4 (v3.7.4:e09359112e, Jul  8 2019, 14:54:52) \n[Clang 6.0 (clang-600.0.57)]'

In [3]:
time.asctime()

'Mon Apr 13 17:02:03 2020'

In [4]:
np.__version__

'1.16.0'

In [5]:
pybel.get_version()

'0.14.6'

In [6]:
pd.__version__

'0.23.4'

In [7]:
scipy.__version__

'1.2.1'

Load the KG

In [8]:
covid = covid19kg.get_graph()

In [9]:
covid.summarize()

Covid19KG v0.0.1-dev
Number of Nodes: 3954
Number of Edges: 9484
Number of Citations: 185
Number of Authors: 950
Network Density: 6.07E-04
Number of Components: 29
Number of Warnings: 0


In [10]:
covid = pybel.struct.summary.node_summary.get_names(covid)

In [11]:
covid_geneset = covid["HGNC"]

Getting pathway genesets out of the databases

In [12]:
PREFIX = "https://raw.githubusercontent.com/pathwayforte/pathway-forte/master/data/gmt_files/"
KEGG_URL = PREFIX + "kegg_geneset_final.gmt"
REACTOME_URL = PREFIX + "wikipathways_geneset_final.gmt"
WIKIPATHWAYS_URL = PREFIX + "reactome_geneset_final.gmt"

In [13]:
import urllib.request
from io import StringIO

def get_genesets(url: str):
    """Return gene sets as a dictionary by downloading a GMT file."""
    response = StringIO(urllib.request.urlopen(url).read().decode('utf-8'))
    
    genesets_dict = {
        line.strip().split("\t")[0]: line.strip().split("\t")[2:]
        for line in response
    }
    return {
        k: v
        for k, v in genesets_dict.items()
        if len(v) >= 10 and len(v) <= 500
    }

In [14]:
kegg_gene_sets = get_genesets(KEGG_URL)
reactome_gene_sets = get_genesets(REACTOME_URL)
wp_gene_sets = get_genesets(WIKIPATHWAYS_URL)

In [15]:
def _prepare_hypergeometric_test(
        query_gene_set,
        pathway_gene_set,
        gene_universe,
):
    """Prepare the matrix for hypergeometric test calculations.

    :param query_gene_set: gene set to test against pathway
    :param pathway_gene_set: pathway gene set
    :param gene_universe: number of HGNC symbols
    :return: 2x2 matrix
    """
    # Cast lists to sets
    if not isinstance(query_gene_set, set):
        query_gene_set = set(query_gene_set)
    if not isinstance(pathway_gene_set, set):
        pathway_gene_set = set(pathway_gene_set)

    # Return matrix to test hyper-geometric test
    return np.array([
        [
            len(query_gene_set.intersection(pathway_gene_set)),
            len(query_gene_set.difference(pathway_gene_set)),
        ],
        [
            len(pathway_gene_set.difference(query_gene_set)),
            gene_universe - len(pathway_gene_set.union(query_gene_set)),
        ],
    ])

def perform_hypergeometric_test(
        genes_to_test,
        pathway_dict,
        gene_universe: int = 41714,
        apply_threshold=False,
        threshold=0.01,
):
    """Perform hypergeometric tests.

    :param genes_to_test: gene set to test against pathway
    :param pathway_dict: pathway name to gene set
    :param gene_universe: number of HGNC symbols
    :param apply_threshold: return only significant pathways
    :param threshold: significance threshold (by default 0.05)
    """
    rows = []
    for (pathway_id, database), pathway_gene_set in pathway_dict.items():
        # Prepare the test table to conduct the fisher test
        test_table = _prepare_hypergeometric_test(genes_to_test, pathway_gene_set, gene_universe)
        # Calculate fisher test (returns tuple of odds ratio and p_value
        p_value = fisher_exact(test_table, alternative='greater')[1]
        rows.append((database, pathway_id, p_value))

    df = pd.DataFrame(rows, columns=['database', 'pathway_id', 'pval'])
    correction_test = multipletests(df.pval, method='fdr_bh')
    df['qval'] = correction_test[1]

    if apply_threshold:
        print('Filtering out pathways with q-values > 0.05 according to fdr_bh')
        df = df[df['qval'] < threshold]

    return df

In [16]:
def preprocess_genesets(dictionary, database):
    return {
        (key, database): value
        for key, value in dictionary.items()
        if len(value) < 200
    }

In [17]:
kegg_results = perform_hypergeometric_test(
    covid_geneset, preprocess_genesets(kegg_gene_sets, 'kegg'), apply_threshold=True
)
reactome_results = perform_hypergeometric_test(
    covid_geneset, preprocess_genesets(reactome_gene_sets, 'reactome'), apply_threshold=True
)
wp_results = perform_hypergeometric_test(
    covid_geneset, preprocess_genesets(wp_gene_sets, 'wikipathways'), apply_threshold=True
)

Filtering out pathways with q-values > 0.05 according to fdr_bh
Filtering out pathways with q-values > 0.05 according to fdr_bh
Filtering out pathways with q-values > 0.05 according to fdr_bh


In [18]:
kegg_results.sort_values(by=['qval'], ascending=True).head()

,database,pathway_id,pval,qval
260,kegg,hsa05164,4.251925e-60,1.262822e-57
157,kegg,hsa04620,7.710119e-45,1.144953e-42
259,kegg,hsa05162,2.755934e-43,2.728375e-41
262,kegg,hsa05168,1.594562e-40,1.183962e-38
158,kegg,hsa04621,1.430967e-38,8.499943e-37


In [19]:
reactome_results.sort_values(by=['qval'], ascending=True).head()

,database,pathway_id,pval,qval
232,reactome,WP1449,3.144748e-41,1.264189e-38
41,reactome,WP2328,5.248340e-30,1.054916e-27
49,reactome,WP530,6.500526e-28,8.710705e-26
26,reactome,WP3865,5.236807e-27,5.262991e-25
315,reactome,WP4217,3.880071e-26,3.119577e-24


In [20]:
wp_results.sort_values(by=['qval'], ascending=True).head()

,database,pathway_id,pval,qval
575,wikipathways,R-HSA-6783783,5.096497e-29,6.824209e-26
587,wikipathways,R-HSA-6785807,2.127442e-28,1.424322e-25
31,wikipathways,R-HSA-168898,2.496199e-25,1.114137e-22
1276,wikipathways,R-HSA-381119,2.420747e-23,8.103450e-21
779,wikipathways,R-HSA-381042,4.140649e-19,1.108866e-16


The results of the enrichment analysis highlight the Influenza signaling pathway (hsa05164) and other immune system-related pathways as the most enriched pathways.